In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from bs4 import BeautifulSoup
import time
import random

In [2]:
def extractStockData(stock_code):
    
    response = requests.get("https://www.stocklore.ai/api/nse_data.csv?q="+stockCode)
    
    if(response.status_code == 200):
        
        a = response.content.decode("utf-8").split("\n")
        a = [i.split(",") for i in a]
        
        price_df = pd.DataFrame(a[1:-1], columns=a[0])
        price_df.iloc[:,1:] = price_df.iloc[:,1:].astype('float')
        price_df['Date'] = pd.to_datetime(price_df['Date'], format='%Y-%m-%d')
        
        return price_df
    
    else:
        print("Error in Stock Price Data Extraction, Check the stock code name and internet connection")    
        

In [60]:
def getSplitData(stock_code, start_year=2008, end_year=2019, scrape_min_interval=45):
    stocks_split_info = pd.DataFrame(columns = ['NseCode', 'Company', 'OldFV', 'NewFV', 'SplitDate'])
    urls = ["https://www.moneycontrol.com/stocks/marketinfo/splits/homebody.php?sel_year="+str(i) for i in range(start_year, end_year+1)]
    for url in urls:
        time.sleep(scrape_min_interval*(random.random()+1))
        res = requests.get(url)
        soup = BeautifulSoup(res.content,'lxml')
        split_table = soup.find_all('table')[-2]
        split_rows = split_table.find_all('tr')
        rows_with_data = split_rows[3:len(split_rows)-3]
        for row in rows_with_data:
            if(row.find('a')):
                try:
                    code = BeautifulSoup(requests.get("https://www.moneycontrol.com"+row.find('a')['href']).content, 'lxml').find('div', class_="FL gry10").get_text().split('|')[1][5:].strip()
                except:
                    code = 'notScraping'
                company = row.find('a').get_text()
                lst = [row.find_all('td')[i].get_text() for i in range(1,4)]
                stocks_split_info = stocks_split_info.append({'NseCode':code,'Company':company, 'OldFV':lst[0], 'NewFV':lst[1], 'SplitDate':lst[2]}, ignore_index=True)
                
        stock_split_df = stocks_split_info[stocks_split_info["NseCode"]==stock_code]
        


In [72]:
def getBonusData(start_year=2008, end_year=2019, sleep_interval=40):
    bonus_issue_info = pd.DataFrame(columns = ['NseCode', 'Company', 'Ratio', 'AnnouncementDate', 'RecordDate', 'Ex-Bonus'])
    urls = ["https://www.moneycontrol.com/stocks/marketinfo/bonus/homebody.php?sel_year="+str(i) for i in range(start_year, end_year+1)]
    for url in urls:
        #time.sleep(sleep_interval*(random.random()+1))
        res = requests.get(url)
        soup = BeautifulSoup(res.content,'lxml')
        bonus_table = soup.find_all('table', class_="b_12 dvdtbl")[0]
        bonus_rows = bonus_table.find_all('tr')
        rows_with_data = bonus_rows[2:]
        for row in rows_with_data:
            if(row.find('a')):
                try:
                    code = BeautifulSoup(requests.get("https://www.moneycontrol.com"+row.find('a')['href']).content, 'lxml').find('div', class_="FL gry10").get_text().split('|')[1][5:].strip()
                except:
                    code = 'notScraping'
                company = row.find('a').get_text()
                lst = [row.find_all('td')[i].get_text() for i in range(1,5)]
                bonus_issue_info = bonus_issue_info.append({'NseCode': code, 'Company':company, 'Ratio':lst[0], 'AnnouncementDate':lst[1], 'RecordDate':lst[2], 'Ex-Bonus':lst[3]}, ignore_index=True)
    return bonus_issue_info       

In [ ]:
def adjust_split(stockCode):
    
    price_df = extractStockData(stockCode)
    
    

In [61]:
stocks_split_info = pd.DataFrame(columns = ['NseCode', 'Company', 'OldFV', 'NewFV', 'SplitDate'])

In [ ]:
a = getBonusData(start_year=2018, end_year=2019)
a

In [63]:
stocks_split_info['NseCode'][2]

'BRITANNIA'

In [69]:

s = BeautifulSoup(requests.get("https://www.moneycontrol.com/india/stockpricequote/autoancillaries/mothersonsumisystems/MSS01").content, 'lxml').find('div', class_="FL gry10").get_text().split('|')[1][5:].strip()
s

'MOTHERSUMI'